In [1]:
import pandas as pd
import numpy as np
import warnings as wn
import datetime as dt
import xgboost as xgb

wn.simplefilter("ignore")

In [2]:
dtypes_auc = {"ref_type_id": np.int8, "source_id": np.int16, "device_id": np.int64 }

dtypes_ins = { "application_id": np.int16, "ref_type":"category", "click_hash":"category", "device_model":"category",
               "device_countrycode":"category", "device_brand": "category", "session_user_agent":"category",
               "kind":"category", "wifi":"category", "device_language":"category"}      

dtypes_clk = { "advertiser_id": np.int8, "action_id": np.float16, "source_id": np.int8, "country_code": "category", 
               "latitude": np.float16, "longitude": np.float16, "wifi_connection": "category", "carrier_id": np.float16,
               "trans_id": "object", "os_major":np.float32, "brand": np.float16, "touchX": np.float16,
               "touchY": np.float16  }

dtypes_evt = { "event_id":np.int32, "application_id": np.int32, "attributed":"category", "device_countrycode":"category",
               "device_os_version":"category", "device_brand": np.float32, "device_model": "category",
               "device_city":"category", "session_user_agent":"category", "user_agent":np.float32, "carrier":"category",
               "kind":"category", "wifi":"category", "connection_type":"category", "device_language":"category",
               "trans_id":"object" }

In [3]:
loc = r"C:\Users\FacundoTorraca\Documents\Facultad\Organizacion de Datos\Trabajo Practico 2"

In [4]:
auc = pd.read_csv( loc + "\\auctions.csv.gzip", compression = "gzip", dtype = dtypes_auc ) 
auc["date"] = pd.to_datetime( auc["date"], errors = "coerce" )

In [5]:
ins = pd.read_csv( loc + '\\installs.csv.gzip', compression = 'gzip', dtype = dtypes_ins )
ins["created"] = pd.to_datetime( ins["created"], errors = "coerce" )

In [6]:
evt = pd.read_csv( loc + '\\events.csv.gzip', compression = "gzip", dtype = dtypes_evt )
evt["date"] = pd.to_datetime( evt["date"], errors = "coerce" )

In [7]:
clk = pd.read_csv( loc + '\\clicks.csv.gzip', compression = "gzip" )
clk["touchX"] = clk["touchX"].fillna(0)
clk["touchY"] = clk["touchY"].fillna(0)
clk["touchX"] = pd.to_numeric( clk["touchX"], downcast = "float", errors = "coerce" )
clk["touchY"] = pd.to_numeric( clk["touchY"], downcast = "float", errors = "coerce" )
clk["created"] = pd.to_datetime( clk["created"], errors = "coerce" )

In [8]:
tar = pd.read_csv( loc + "\\target_final_competencia_revamped.csv" )

### Acomodo los FEATURES para cada categoria

In [9]:
ins_1 = ins.loc[ ins['created'].dt.day.between(18,20) ]
ins_2 = ins.loc[ ins['created'].dt.day.between(19,21) ]
ins_3 = ins.loc[ ins['created'].dt.day.between(20,22) ]
ins_4 = ins.loc[ ins['created'].dt.day.between(21,23) ]
ins_5 = ins.loc[ ins['created'].dt.day.between(22,24) ]
ins_6 = ins.loc[ ins['created'].dt.day.between(23,25) ]
ins_7 = ins.loc[ ins['created'].dt.day.between(24,26) ]

In [13]:
clk_1 = clk.loc[ clk['created'].dt.day.between(18,20) ]
clk_2 = clk.loc[ clk['created'].dt.day.between(19,21) ]
clk_3 = clk.loc[ clk['created'].dt.day.between(20,22) ]
clk_4 = clk.loc[ clk['created'].dt.day.between(21,23) ]
clk_5 = clk.loc[ clk['created'].dt.day.between(22,24) ]
clk_6 = clk.loc[ clk['created'].dt.day.between(23,25) ]
clk_7 = clk.loc[ clk['created'].dt.day.between(24,26) ]

In [14]:
evt_1 = evt.loc[ evt['date'].dt.day.between(18,20) ]
evt_2 = evt.loc[ evt['date'].dt.day.between(19,21) ]
evt_3 = evt.loc[ evt['date'].dt.day.between(20,22) ]
evt_4 = evt.loc[ evt['date'].dt.day.between(21,23) ]
evt_5 = evt.loc[ evt['date'].dt.day.between(22,24) ]
evt_6 = evt.loc[ evt['date'].dt.day.between(23,25) ]
evt_7 = evt.loc[ evt['date'].dt.day.between(24,26) ]

In [11]:
auc_1 = auc.loc[ auc['date'].dt.day.between(18,20) ]
auc_2 = auc.loc[ auc['date'].dt.day.between(19,21) ]
auc_3 = auc.loc[ auc['date'].dt.day.between(20,22) ]
auc_4 = auc.loc[ auc['date'].dt.day.between(21,23) ]
auc_5 = auc.loc[ auc['date'].dt.day.between(22,24) ]
auc_6 = auc.loc[ auc['date'].dt.day.between(23,25) ]
auc_7 = auc.loc[ auc['date'].dt.day.between(24,26) ]

### Acomodo Features de INSTALLS & AUCTIONS para prediccion

In [28]:
ins_1_pre = ins_1[ ["ref_hash", "created"] ]
ins_2_pre = ins_2[ ["ref_hash", "created"] ]
ins_3_pre = ins_3[ ["ref_hash", "created"] ]
ins_4_pre = ins_4[ ["ref_hash", "created"] ]
ins_5_pre = ins_5[ ["ref_hash", "created"] ]
ins_6_pre = ins_6[ ["ref_hash", "created"] ]
ins_7_pre = ins_7[ ["ref_hash", "created"] ]

ins_1_pre["_sc"] = ins_1_pre["created"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_1_pre["_sc"] = ins_1_pre["_sc"].dt.total_seconds(); del( ins_1_pre["created"] )
ins_2_pre["_sc"] = ins_2_pre["created"] - dt.datetime( year = 2019, month = 4, day = 19 ); ins_2_pre["_sc"] = ins_2_pre["_sc"].dt.total_seconds(); del( ins_2_pre["created"] )
ins_3_pre["_sc"] = ins_3_pre["created"] - dt.datetime( year = 2019, month = 4, day = 20 ); ins_3_pre["_sc"] = ins_3_pre["_sc"].dt.total_seconds(); del( ins_3_pre["created"] )
ins_4_pre["_sc"] = ins_4_pre["created"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_4_pre["_sc"] = ins_4_pre["_sc"].dt.total_seconds(); del( ins_4_pre["created"] )
ins_5_pre["_sc"] = ins_5_pre["created"] - dt.datetime( year = 2019, month = 4, day = 22 ); ins_5_pre["_sc"] = ins_5_pre["_sc"].dt.total_seconds(); del( ins_5_pre["created"] )
ins_6_pre["_sc"] = ins_6_pre["created"] - dt.datetime( year = 2019, month = 4, day = 23 ); ins_6_pre["_sc"] = ins_6_pre["_sc"].dt.total_seconds(); del( ins_6_pre["created"] )
ins_7_pre["_sc"] = ins_7_pre["created"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_7_pre["_sc"] = ins_7_pre["_sc"].dt.total_seconds(); del( ins_7_pre["created"] )

In [ ]:
auc_1_pre = auc_1[ ["device_id", "date"] ]
auc_2_pre = auc_2[ ["device_id", "date"] ]
auc_3_pre = auc_3[ ["device_id", "date"] ]
auc_4_pre = auc_4[ ["device_id", "date"] ]
auc_5_pre = auc_5[ ["device_id", "date"] ]
auc_6_pre = auc_6[ ["device_id", "date"] ]
auc_7_pre = auc_7[ ["device_id", "date"] ]

auc_1_pre["_st"] = auc_1_pre["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_1["_st"] = auc_1_pre["_st"].dt.total_seconds(); del( auc_1_pre["date"] )
auc_2_pre["_st"] = auc_2_pre["date"] - dt.datetime( year = 2019, month = 4, day = 19 ); auc_2["_st"] = auc_2_pre["_st"].dt.total_seconds(); del( auc_2_pre["date"] )
auc_3_pre["_st"] = auc_3_pre["date"] - dt.datetime( year = 2019, month = 4, day = 20 ); auc_3["_st"] = auc_3_pre["_st"].dt.total_seconds(); del( auc_3_pre["date"] )
auc_4_pre["_st"] = auc_4_pre["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_4["_st"] = auc_4_pre["_st"].dt.total_seconds(); del( auc_4_pre["date"] )
auc_5_pre["_st"] = auc_5_pre["date"] - dt.datetime( year = 2019, month = 4, day = 22 ); auc_5["_st"] = auc_5_pre["_st"].dt.total_seconds(); del( auc_5_pre["date"] )
auc_6_pre["_st"] = auc_6_pre["date"] - dt.datetime( year = 2019, month = 4, day = 23 ); auc_6["_st"] = auc_6_pre["_st"].dt.total_seconds(); del( auc_6_pre["date"] )
auc_7_pre["_st"] = auc_7_pre["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_7["_st"] = auc_7_pre["_st"].dt.total_seconds(); del( auc_7_pre["date"] )

### Agregamos los LABEL´S

In [30]:
ins_1["_sc"] = ins_1_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_2["_sc"] = ins_2_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_3["_sc"] = ins_3_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_4["_sc"] = ins_4_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_5["_sc"] = ins_5_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_6["_sc"] = ins_6_pre.groupby( by = "ref_hash" ).transform( "min" )
ins_7["_sc"] = ins_7_pre.groupby( by = "ref_hash" ).transform( "min" )

In [36]:
auc_1["_st"] = auc_1_pre.groupby( by = "device_id" ).transform( "min" )
auc_2["_st"] = auc_2_pre.groupby( by = "device_id" ).transform( "min" )
auc_3["_st"] = auc_3_pre.groupby( by = "device_id" ).transform( "min" )
auc_4["_st"] = auc_4_pre.groupby( by = "device_id" ).transform( "min" )
auc_5["_st"] = auc_5_pre.groupby( by = "device_id" ).transform( "min" )
auc_6["_st"] = auc_6_pre.groupby( by = "device_id" ).transform( "min" )
auc_7["_st"] = auc_7_pre.groupby( by = "device_id" ).transform( "min" )

In [37]:
auc_1

,date,device_id,ref_type_id,source_id,_sp,_st
221726,2019-04-20 23:57:27.912838,1109595589636746168,7,0,259047.912838,0 days 00:00:38.281742
221727,2019-04-20 23:57:28.381114,5896614299191635403,1,0,259048.381114,2 days 01:46:34.133499
221728,2019-04-20 23:57:28.515423,4172466725848941608,1,0,259048.515423,0 days 06:27:50.996158
221729,2019-04-20 23:57:28.700884,2616279795187318849,7,0,259048.700884,0 days 04:02:46.305613
221730,2019-04-20 23:57:28.868312,8034952072073026056,1,0,259048.868312,0 days 00:35:31.040530
221731,2019-04-20 23:57:29.202778,8018738201130876812,1,0,259049.202778,0 days 00:00:03.433616
221732,2019-04-20 23:57:29.441246,4914962826064649332,7,0,259049.441246,0 days 07:12:58.105372
221733,2019-04-20 23:57:29.644078,3249069566262483358,1,0,259049.644078,2 days 23:18:50.529638
221734,2019-04-20 23:57:29.896508,3352762403977995643,1,0,259049.896508,0 days 05:50:20.094461
221735,2019-04-20 23:57:30.453524,9003488718010194664,1,0,259050.453524,1 days 02:02:41.468019
